In [1]:
import json
import pandas as pd

## Loading

In [2]:
tweet_file = "../../data/tweets/all_tweets_with_coding.json"
tweets_df = pd.read_json(tweet_file)
tweets_df = tweets_df.join(tweets_df.affects.apply(pd.Series))
tweets_df = tweets_df.drop(["affects"], axis = 1)

mass_pol_file = "../../data/polarization/processed_data/state_mass_pol.csv"
mass_pol_df = pd.read_csv(mass_pol_file)

elite_pol_file = "../../data/polarization/processed_data/state_elite_pol.csv"
elite_pol_df = pd.read_csv(elite_pol_file)

demographics_file = "../../data/demographics/ACS2014_5yr.csv"
demographics_df = pd.read_csv(demographics_file)
young_demo_df = demographics_df[["DP05_0008PE", "DP05_0009PE", "DP05_0010PE"]][1:].astype(float).sum(axis=1).to_frame()
old_demo_df = demographics_df[["DP05_0011PE", "DP05_0012PE", "DP05_0013PE", "DP05_0014PE", "DP05_0015PE", "DP05_0016PE"]][1:].astype(float).sum(axis=1).to_frame()
demo_df = young_demo_df / old_demo_df
demo_df = demographics_df[["NAME"]][1:].join(demo_df)
demo_df = demo_df.rename(columns={"NAME": "state", 0: "percentAge20To44"})

## Merge

In [3]:
final_dataset_df = tweets_df.drop(['created_at', 'text', 'in_reply_to_user_id', 'mentioned_url', 'photo_url'], axis = 1)
final_dataset_df = final_dataset_df.merge(mass_pol_df, on="state", how="left")
final_dataset_df = final_dataset_df.merge(elite_pol_df, on="state", how="left")
final_dataset_df = final_dataset_df.merge(demo_df, on="state", how="left")
final_dataset_df

,user,favorite_count,retweet_count,follower_count,is_attack,state,gender,party,incumbency,competitiveness,...,positive,sadness,trust,disgust,anger,joy,ideoConsist,h_dist,s_dist,percentAge20To44
0,asahutchinson,3,9,62445,False,Arkansas,1,R,False,False,...,1,0,1,1,1,0,8.0,0.8441295943400905,0.5454651814419283,0.798526
1,drichardsonor,1,0,2160,False,Oregon,1,R,False,False,...,3,0,2,0,0,1,13.0,1.0495518594687063,1.1823648401757914,0.792857
2,robastorino,9,11,17198,False,New York,1,R,False,False,...,1,0,1,0,0,1,10.0,0.83222601932794,1.02074743579523,0.843137
3,french4freedom,2,1,133,False,South Carolina,1,L,False,False,...,1,1,0,0,1,0,5.0,0.8459304871381109,1.028572448626407,0.787952
4,rickettsforgov,3,4,3200,False,Nebraska,1,R,False,False,...,1,0,1,0,0,0,10.0,nan,0.9868067400072734,0.827848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28037,hickenlooper,5,8,162908,False,Colorado,1,D,True,True,...,2,0,1,1,1,1,9.0,2.077331031627843,1.7270138591005322,0.934211
28038,senator_hatch,3,2,3162,False,Iowa,1,D,False,False,...,1,0,0,0,1,0,9.5,0.9343941597599107,0.998577546059275,0.755981
28039,onetoughnerd,3,1,65522,False,Michigan,1,R,True,True,...,2,0,2,0,0,1,10.0,1.043946499475657,1.076631440157968,0.737089
28040,scottwalker,6,3,281967,False,Wisconsin,1,R,True,True,...,0,0,0,0,0,0,8.0,1.167701041132485,1.1071669655356562,0.754137


## Export

In [4]:
final_dataset_df.to_csv("../../data/final_dataset.csv")